## Prep

In [ ]:
!pip install -U kaleido plotly

In [ ]:
!pip install ray 

In [ ]:
!pip install fairlearn

In [ ]:
!pip install cma

In [ ]:
!pip install dalex -U

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Helpers

In [ ]:
import math
import numpy as np
import pandas as pd
import sys
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score
from scipy.stats import t
from scipy.optimize import minimize # The black-box optimization algorithm used to find a candidate solution
from numba import jit       # Just-in-Time (JIT) compiler to accelerate Python code
# import ray 
import timeit               # To time the execution of ours experiments
import cma
from time import time
import warnings

np.set_printoptions(precision=5, suppress=True)


# This function returns the inverse of Student's t CDF using the degrees of freedom in nu for the corresponding
# probabilities in p. It is a Python implementation of Matlab's tinv function: https://www.mathworks.com/help/stats/tinv.html
def tinv(p, nu):
    return t.ppf(p, nu)


# This function computes the sample standard deviation of the vector v, with Bessel's correction
def stddev(v):
    n = v.size
    variance = (np.var(v) * n) / (n-1) # Variance with Bessel's correction
    return np.sqrt(variance)           # Compute the standard deviation


# This function computes a (1-delta)-confidence upper bound on the expected value of a random
# variable using Student's t-test. It analyzes the data in v, which holds i.i.d. samples of the random variable.
# The upper confidence bound is given by 
#    sampleMean + sampleStandardDeviation/sqrt(n) * tinv(1-delta, n-1),
#    where n is the number of points in v.
def ttestUpperBound(v, delta):	
    n  = v.size
    res = v.mean() + stddev(v) / math.sqrt(n) * tinv(1.0 - delta, n - 1)
    return res


# This function works similarly to ttestUpperBound, but returns a conservative upper bound. It uses 
# data in the vector v (i.i.d. samples of a random variable) to compute the relevant statistics 
# (mean and standard deviation) but assumes that the number of points being analyzed is k instead of |v|.
# This function is used to estimate what the output of ttestUpperBound would be if it were to
# be run on a new vector, v, containing values sampled from the same distribution as
# the points in v. The 2.0 factor in the calculation is used to double the width of the confidence interval,
# when predicting the outcome of the safety test, in order to make the algorithm less confident/more conservative.
def predictTTestUpperBound(v, delta, k):
    # conservative prediction of what the upper bound will be in the safety test for the a given constraint
    res = v.mean() + 2.0 * stddev(v) / math.sqrt(k) * tinv(1.0 - delta, k - 1)
    return res


def calc_fp_sklearn(X, Y, clf):
  y_pred = clf.predict(X)
  tn, fp, fn, tp = confusion_matrix(Y, y_pred).ravel()
  return fp / (fp + tn)
  
def calc_odds_sklearn(X, Y, clf):
	x_male = X[X[:, 0] == 1]
	y_pred_male = clf.predict(x_male)
	y_male = Y[X[:, 0] == 1]

	x_female = X[X[:, 0] == 0]
	y_pred_female = clf.predict(x_female)
	y_female = Y[X[:, 0] == 0]

	tnr_male, fpr_male, fnr_male, tpr_male = confusion_matrix(y_male, y_pred_male, normalize="true").ravel()
	tnr_female, fpr_female, fnr_female, tpr_female = confusion_matrix(y_female, y_pred_female, normalize="true").ravel()

	return max(abs(fpr_female - fpr_male), abs(tpr_female - tpr_male))
 
def calc_eq_op_sklearn(X, Y, clf):
	x_male = X[X[:, 0] == 1]
	y_pred_male = clf.predict(x_male)
	y_male = Y[X[:, 0] == 1]

	x_female = X[X[:, 0] == 0]
	y_pred_female = clf.predict(x_female)
	y_female = Y[X[:, 0] == 0]

	tnr_male, fpr_male, fnr_male, tpr_male = confusion_matrix(y_male, y_pred_male, normalize="true").ravel()
	tnr_female, fpr_female, fnr_female, tpr_female = confusion_matrix(y_female, y_pred_female, normalize="true").ravel()

	return abs(fnr_female - fnr_male)

def calc_demographic_parity_sklearn(X, Y, clf):
  y_pred = clf.predict(X)
  return y_pred.sum() / X.shape[0]


def cmaes_minimize(evalf, n_features, sigma0=0.0001, restarts=20, n_iters=1000, theta0=None):
		has_theta0 = not(theta0 is None)
		np.random.seed(np.floor(100000*(time()%10)).astype(int))
		theta_opt = None
		val_min = np.inf

		theta0 = theta0 if not(theta0 is None) else np.zeros(n_features)
		next_theta0 = theta0.copy()
		for _ in range(restarts):
			options = {'verb_log':0, 'verbose':-9, 'verb_disp':0, 'tolfun':1e-12, 'seed':make_seed(), 'maxiter':n_iters}
			es = cma.CMAEvolutionStrategy(next_theta0, sigma0, options)
			with warnings.catch_warnings():				
				warnings.simplefilter('ignore', category=RuntimeWarning)
				es.optimize(evalf)
			theta = es.result.xbest
			if not(theta is None):
				value = evalf(theta)
				if value < val_min:
					theta_opt = theta.copy()
					val_min = value
			next_theta0 = theta0 + np.random.normal(size=theta0.shape)
		return { 'x': theta_opt }


def make_seed(digits=8, random_state=np.random):
    return np.floor(random_state.rand()*10**digits).astype(int)

## Seldonian Algorithm

In [ ]:
# Generate numPoints data points
def generateData(numPoints):
    X = pd.DataFrame([
                np.random.binomial(size=numPoints, n = 1, p = 0.3),
                np.random.normal(0.0, 1.0, numPoints),
            ]).T.values # Sample x from a standard normal distribution

    intercept = (np.random.rand(1))[0]
    z = intercept
    coefs = np.random.rand(X.shape[1]) * 10
    z = z + np.dot(coefs, X.T)
    Y = np.round(1/(1+np.exp(-z))) # Set y to be x, plus noise from a standard normal distribution
    print('Original params: ', intercept, coefs)
    return (X,Y)


# Uses the weights in theta to predict the output value, y, associated with the provided x.
# This function assumes we are performing linear regression, so that theta has
# two elements: the y-intercept (first parameter) and slope (second parameter)
def predict(theta, x):
    return 1/(1 + np.exp(-np.dot(theta, x)))

def logistic_cost(y_i, y_hat):
    return -y_i*np.log(y_hat) - (1-y_i)*np.log(1-y_hat)

# Estimator of the primary objective, in this case, the negative sample mean squared error
def fHat(theta, X, Y):
    n = X.shape[0]       # Number of points in the data set
    res = 0.0           # Used to store the sample MSE we are computing
    for i in range(n):  # For each point X[i] in the data set ...
        prediction = predict(theta, [1, *X[i, :]])  # Get the prediction using theta
        prediction = np.clip(prediction, 1e-5, 1 - 1e-5)
        cost = logistic_cost(Y[i], prediction) # Add the squared error to the result
        res += cost

    res /= n            # Divide by the number of points to obtain the sample mean squared error
    return -res         # Returns the negative sample mean squared error


# Returns unbiased estimates of g_1(theta), computed using the provided data
def g_max_loss(maximum = 0.1):
  def _(theta, X, Y):
    n = X.shape[0]       # Number of points in the data set
    res = np.zeros(n)   # We will get one estimate per point; initialize res to store these estimates
    for i in range(n):  # For each point X[i] in the data set ...
        prediction = predict(theta, [1, *X[i, :]])  # Get the prediction using theta
        prediction = np.clip(prediction, 1e-5, 1 - 1e-5)
        res[i] = logistic_cost(Y[i], prediction)  # Compute the squared error for the i-th data point'
    res = res - maximum     # We want the MSE to be less than 2.0, so g(theta) = MSE-2.0
    return pd.Series(res).dropna().values
  return _


# Returns unbiased estimates of g_2(theta), computed using the provided data
def g_min_loss(minimum = 1e-12):
  def _(theta, X, Y):
    n = X.shape[0]      # Number of points in the data set
    res = np.zeros(n)   # We will get one estimate per point; initialize res to store these estimates
    for i in range(n):  # For each point X[i] in the data set ...
        prediction = predict(theta, [1, *X[i, :]])  # Get the prediction using theta
        res[i] = logistic_cost(Y[i], prediction)  # Compute the squared error for the i-th data point
    res = minimum - res    # We want the MSE to be at least 1.25, so g(theta) = 1.25-MSE
    return pd.Series(res).dropna().values
  return _


def calc_f1(X, Y, predict_prob):
  y_pred = []
  for (data, y) in zip(X, Y):
    prediction = predict_prob([1, *data])  # Get the prediction using theta
    prediction_rounded = round(prediction)
    y_pred.append(prediction_rounded)

  return f1_score(Y, y_pred, average='macro')

def g_min_f1(minimum = 0.5, batch_size = 100):
  def _(theta, X, Y):
    n = X.shape[0]      # Number of points in the data set
    num_batches = int(np.floor(n / batch_size))
    res = np.array([])
    indices = np.array(range(n))
    
    def predict_prob(x):
      return predict(theta, x)

    for batch in get_batch(indices, num_batches):
        res = np.append(res, calc_f1(X[batch, :], Y[batch], predict_prob))
    res = minimum - res
    return pd.Series(res).dropna().values
  return _


def calc_fp_diff(X, Y, predict_prob):
    male_X = X[X[:, 0] == 1,:]
    female_X = X[X[:, 0] == 0,:]
    male_Y = Y[X[:, 0] == 1]
    female_Y = Y[X[:, 0] == 0]
    
    male_pred_Y = predict_prob(np.hstack((np.ones(male_X.shape[0]).reshape((male_X.shape[0],1)), male_X)))
    male_pred_Y = np.round(male_pred_Y)

    male_tn, male_fp, male_fn, male_tp = confusion_matrix(male_Y, male_pred_Y.T, labels=[0,1]).ravel()

    female_pred_Y = predict_prob(
        np.hstack(
          (
            np.ones(female_X.shape[0]).reshape((female_X.shape[0],1)), 
            female_X
          )
        )
    )
    female_pred_Y = np.round(female_pred_Y)
    female_tn, female_fp, female_fn, female_tp = confusion_matrix(female_Y, female_pred_Y.T, labels=[0,1]).ravel()

    # https://stackoverflow.com/a/31324768/6008808
    fp_male = male_fp / (male_fp + male_tn) if male_fp + male_tn > 0 else 0
    fp_female = female_fp / (female_fp + female_tn) if female_fp + female_tn > 0 else 0

    return np.abs(fp_female - fp_male)

# https://stackoverflow.com/a/8290508/6008808
def get_batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]


def g_fp_diff(
    maximum = 0.05,
    batch_size = 10
):
  def _(theta, X, Y):
    n = X.shape[0]      # Number of points in the data set
    res = np.array([])
    
    def predict_prob(x):
      return predict(x, theta)

    batches = []
    while len(batches) < n:
      train_x, _, train_y, _ = train_test_split(X, Y, train_size=batch_size, random_state=len(batches))
      batches.append((train_x, train_y))

    for x, y in batches:
      res = np.append(res, calc_fp_diff(x, y, predict_prob))

    res = res - maximum
    return pd.Series(res).dropna().values
  return _

def calc_eq_odds_diff(X, Y, predict_prob):
  male_X = X[X[:, 0] == 1,:]
  female_X = X[X[:, 0] == 0,:]
  male_Y = Y[X[:, 0] == 1]
  female_Y = Y[X[:, 0] == 0]

  male_pred_Y = predict_prob(np.hstack((np.ones(male_X.shape[0]).reshape((male_X.shape[0],1)), male_X)))
  male_pred_Y = np.round(male_pred_Y)

  tn_male, fp_male, fn_male, tp_male = confusion_matrix(male_Y, male_pred_Y.T, labels=[0,1]).ravel()

  female_pred_Y = predict_prob(np.hstack((np.ones(female_X.shape[0]).reshape((female_X.shape[0],1)), female_X)))
  female_pred_Y = np.round(female_pred_Y)
  tn_female, fp_female, fn_female, tp_female = confusion_matrix(female_Y, female_pred_Y.T, labels=[0,1]).ravel()

  fpr_male = fp_male / (fp_male + tn_male) if (fp_male + tn_male) > 0 else 0
  fpr_female = fp_female / (fp_female + tn_female) if (fp_female + tn_female) > 0 else 0

  tpr_male = tp_male / (tp_male + fn_male) if (tp_male + fn_male) > 0 else 0
  tpr_female = tp_female / (tp_female + fn_female) if (tp_female + fn_female) > 0 else 0

  return max(abs(fpr_female - fpr_male), abs(tpr_female - tpr_male))


def g_eq_odds_diff(
    maximum = 0.1,
    batch_size = 100,
    bootstrap_num = 500
):
  def _(theta, X, Y):
    # n = X.shape[0]
    res = np.array([])
    
    def predict_prob(x):
      return predict(x, theta)

    batches = []
    while len(batches) < bootstrap_num:
      train_x, _, train_y, _ = train_test_split(X, Y, train_size=batch_size, random_state=len(batches))
      batches.append((train_x, train_y))

    for x, y in batches:
      res = np.append(res, calc_eq_odds_diff(x, y, predict_prob))

    res = res - maximum
    return pd.Series(res).dropna().values
  return _


def calc_eq_op_diff(X, Y, predict_prob):
  male_X = X[X[:, 0] == 1,:]
  female_X = X[X[:, 0] == 0,:]
  male_Y = Y[X[:, 0] == 1]
  female_Y = Y[X[:, 0] == 0]

  male_pred_Y = predict_prob(np.hstack((np.ones(male_X.shape[0]).reshape((male_X.shape[0],1)), male_X)))
  male_pred_Y = np.round(male_pred_Y)

  tnr_male, fpr_male, fnr_male, tpr_male = confusion_matrix(male_Y, male_pred_Y.T, labels=[0,1], normalize='true').ravel()

  female_pred_Y = predict_prob(np.hstack((np.ones(female_X.shape[0]).reshape((female_X.shape[0],1)), female_X)))
  female_pred_Y = np.round(female_pred_Y)
  tnr_female, fpr_female, fnr_female, tpr_female = confusion_matrix(female_Y, female_pred_Y.T, labels=[0,1], normalize='true').ravel()

  return abs(fnr_female - fnr_male)


def g_eq_op_diff(
    maximum = 0.1,
    batch_size = 100,
    bootstrap_num = 500
):
  def _(theta, X, Y):
    # n = X.shape[0]
    res = np.array([])
    
    def predict_prob(x):
      return predict(x, theta)

    batches = []
    while len(batches) < bootstrap_num:
      train_x, _, train_y, _ = train_test_split(X, Y, train_size=batch_size, random_state=len(batches))
      batches.append((train_x, train_y))

    for x, y in batches:
      res = np.append(res, calc_eq_op_diff(x, y, predict_prob))

    res = res - maximum
    return pd.Series(res).dropna().values
  return _


def get_logistic_regression_initial_params(X, Y):
    reg = LogisticRegression(max_iter=10000).fit(X, Y)
    theta0 = reg.intercept_[0]   # Gets theta0, the y-intercept coefficient
    thetas = np.array([theta0, *reg.coef_[0]])
    print("initial params", thetas)
    return thetas

# Our Quasi-Seldonian linear regression algorithm operating over data (X,Y).
# The pair of objects returned by QSA is the solution (first element) 
# and a Boolean flag indicating whether a solution was found (second element).
def QSA(X, Y, gHats, deltas):
    # Put 75% of the data in candidateData (D1), and the rest in safetyData (D2)
    candidateData_len = 0.75
    candidateData_X, safetyData_X, candidateData_Y, safetyData_Y = train_test_split(
                                X, Y, test_size=1-candidateData_len, shuffle=False)

    # Get the candidate solution
    candidateSolution = getCandidateSolution(candidateData_X, candidateData_Y, gHats, deltas, safetyData_X.size)

    # Run the safety test
    passedSafety      = safetyTest(candidateSolution, safetyData_X, safetyData_Y, gHats, deltas)

    # Return the result and success flag
    return [candidateSolution, passedSafety]


# Run the safety test on a candidate solution. Returns true if the test is passed.
#   candidateSolution: the solution to test. 
#   (safetyData_X, safetyData_Y): data set D2 to be used in the safety test.
#   (gHats, deltas): vectors containing the behavioral constraints and confidence levels.
def safetyTest(candidateSolution, safetyData_X, safetyData_Y, gHats, deltas):

    for i in range(len(gHats)):	# Loop over behavioral constraints, checking each
        g         = gHats[i]	# The current behavioral constraint being checked
        delta     = deltas[i]	# The confidence level of the constraint

        # This is a vector of unbiased estimates of g(candidateSolution)
        g_samples = g(candidateSolution, safetyData_X, safetyData_Y) 

        # Check if the i-th behavioral constraint is satisfied
        upperBound = ttestUpperBound(g_samples, delta) 

        print('safety', upperBound)

        if upperBound > 0.0: # If the current constraint was not satisfied, the safety test failed
            return False

    # If we get here, all of the behavioral constraints were satisfied			
    return True


# The objective function maximized by getCandidateSolution.
#     thetaToEvaluate: the candidate solution to evaluate.
#     (candidateData_X, candidateData_Y): the data set D1 used to evaluated the solution.
#     (gHats, deltas): vectors containing the behavioral constraints and confidence levels.
#     safetyDataSize: |D2|, used when computing the conservative upper bound on each behavioral constraint.
def candidateObjective(thetaToEvaluate, candidateData_X, candidateData_Y, gHats, deltas, safetyDataSize):	

    # Get the primary objective of the solution, fHat(thetaToEvaluate)
    result = fHat(thetaToEvaluate, candidateData_X, candidateData_Y)
    predictSafetyTest = True     # Prediction of what the safety test will return. Initialized to "True" = pass
    for i in range(len(gHats)):  # Loop over behavioral constraints, checking each
        g         = gHats[i]       # The current behavioral constraint being checked
        delta     = deltas[i]      # The confidence level of the constraint

        # This is a vector of unbiased estimates of g_i(thetaToEvaluate)
        g_samples = g(thetaToEvaluate, candidateData_X, candidateData_Y)

        # Get the conservative prediction of what the upper bound on g_i(thetaToEvaluate) will be in the safety test
        upperBound = predictTTestUpperBound(g_samples, delta, safetyDataSize)

        print('candidate', upperBound)
        
        # We don't think the i-th constraint will pass the safety test if we return this candidate solution
        if upperBound > 0.0:

            if predictSafetyTest:
                # Set this flag to indicate that we don't think the safety test will pass
                predictSafetyTest = False  

                # Put a barrier in the objective. Any solution that we think will fail the safety test will have a
                # large negative performance associated with it
                result = -100000.0    

            # Add a shaping to the objective function that will push the search toward solutions that will pass 
            # the prediction of the safety test
            result = result - upperBound
    
    # Negative because our optimizer (Powell) is a minimizer, but we want to maximize the candidate objective
    return -result  


# Use the provided data to get a candidate solution expected to pass the safety test.
#    (candidateData_X, candidateData_Y): data used to compute a candidate solution.
#    (gHats, deltas): vectors containing the behavioral constraints and confidence levels.
#    safetyDataSize: |D2|, used when computing the conservative upper bound on each behavioral constraint.
def getCandidateSolution(candidateData_X, candidateData_Y, gHats, deltas, safetyDataSize):

    # Chooses the black-box optimizer we will use (Powell)
    minimizer_method = 'Powell'
    minimizer_options={'disp': True, 'maxiter': 1000}

    # Initial solution given to Powell: simple linear fit we'd get from ordinary least squares linear regression
    initialSolution  = get_logistic_regression_initial_params(candidateData_X, candidateData_Y)

    # Use Powell to get a candidate solution that tries to maximize candidateObjective
    res = minimize(candidateObjective, x0=initialSolution, method=minimizer_method, options=minimizer_options,
        args=(candidateData_X, candidateData_Y, gHats, deltas, safetyDataSize))
    
    def eval_func(theta):
      return candidateObjective(theta, candidateData_X, candidateData_Y, gHats, deltas, safetyDataSize)

    # res = cmaes_minimize(evalf=eval_func, n_features=candidateData_X.shape[1] + 1, theta0=initialSolution)

    # Return the candidate solution we believe will pass the safety test
    return res.x

## Playground

In [ ]:
X,Y = generateData(10000)
n = X.shape[0]
res = 0.0
for i in range(n):  # For each point X[i] in the data set ...
  prediction = predict(np.array([1,2,3,4,5]), [1, *X[i, :]])  # Get the prediction using theta
  res += logistic_cost(Y[i], prediction) # Add the squared error to the result
res /= n            # Divide by the number of points to obtain the sample mean squared error
print(res)

In [ ]:
calc_fp_diff([12, 1.14, 7.52], X, Y)

In [ ]:
np.mean(gHat3([12, 1.14, 7.52], X, Y))

In [ ]:
X[[1,2,3],:]

In [ ]:
gHat3([1,2,3], X, Y)

In [ ]:
X = pd.DataFrame([
            np.random.binomial(size=numPoints, n=1, p= 0.5),
            np.random.normal(0.0, 1.0, numPoints), 
        ]).T.values # Sample x from a standard normal distribution

z = (np.random.rand(1))[0]
coefs = np.random.rand(X.shape[1]) * 100
print(coefs.shape)
print(X.shape)
z = z + np.dot(np.random.rand(X.shape[1]) * 100,  X.T)
Y = np.round(1/(1+np.exp(-z))) # Set y to be x, plus noise from a standard normal distribution

In [ ]:
2/3

## Experiment setup

In [ ]:
@ray.remote
def run_experiments(worker_id, nWorkers, ms, numM, numTrials, mTest):

    # Results of the Seldonian algorithm runs
    seldonian_solutions_found = np.zeros((numTrials, numM)) # Stores whether a solution was found (1=True,0=False)
    seldonian_failures_g1     = np.zeros((numTrials, numM)) # Stores whether solution was unsafe, (1=True,0=False), for the 1st constraint, g_1
    seldonian_failures_g2     = np.zeros((numTrials, numM)) # Stores whether solution was unsafe, (1=True,0=False), for the 2nd constraint, g_2
    seldonian_failures_g3     = np.zeros((numTrials, numM)) # Stores whether solution was unsafe, (1=True,0=False), for the 2nd constraint, g_2
    seldonian_fs              = np.zeros((numTrials, numM)) # Stores the primary objective values (fHat) if a solution was found

    # Results of the Least-Squares (LS) linear regression runs
    LS_solutions_found = np.ones((numTrials, numM))  # Stores whether a solution was found. These will all be true (=1)
    LS_failures_g1     = np.zeros((numTrials, numM)) # Stores whether solution was unsafe, (1=True,0=False), for the 1st constraint, g_1
    LS_failures_g2     = np.zeros((numTrials, numM)) # Stores whether solution was unsafe, (1=True,0=False), for the 2nd constraint, g_2
    LS_failures_g3     = np.zeros((numTrials, numM)) # Stores whether solution was unsafe, (1=True,0=False), for the 2nd constraint, g_2
    LS_fs              = np.zeros((numTrials, numM)) # Stores the primary objective values (f) if a solution was found

    # Prepares file where experiment results will be saved
    experiment_number = worker_id
    bin_path = './experiment_results/bin/'
    outputFile = bin_path + 'results%d.npz' % experiment_number
    print("Writing output to", outputFile)

    # Generate the data used to evaluate the primary objective and failure rates
    np.random.seed( (experiment_number+1) * 9999 )
    (testX, testY) = generateData(mTest) 

    for trial in range(numTrials):
        for (mIndex, m) in enumerate(ms):

            # Generate the training data, D
            base_seed         = (experiment_number * numTrials)+1
            np.random.seed(base_seed+trial) # done to obtain common random numbers for all values of m			
            (trainX, trainY)  = generateData(m)

            # Run the Quasi-Seldonian algorithm
            (result, passedSafetyTest) = QSA(trainX, trainY, gHats, deltas)
            if passedSafetyTest:
                seldonian_solutions_found[trial, mIndex] = 1
                trueLoss = -fHat(result, testX, testY)                               # Get the "true" mean squared error using the testData
                seldonian_failures_g1[trial, mIndex] = 1 if trueLoss > 0.05  else 0   # Check if the first behavioral constraint was violated
                seldonian_failures_g2[trial, mIndex] = 1 if trueLoss < 1 else 0 # Check if the second behavioral constraint was violated
                seldonian_failures_g3[trial, mIndex] = 1 if calc_fp_diff(result, testX, testY) < 0.08 else 0
                
                seldonian_fs[trial, mIndex] = -trueLoss                              # Store the "true" negative mean-squared error
                print(f"[(worker {worker_id}/{nWorkers}) Seldonian trial {trial+1}/{numTrials}, m {m}] A solution was found: [{result[0]:.10f}, {result[1]:.10f}]\tfHat over test data: {trueMSE:.10f}")
            else:
                seldonian_solutions_found[trial, mIndex] = 0             # A solution was not found
                seldonian_failures_g1[trial, mIndex]     = 0             # Returning NSF means the first constraint was not violated
                seldonian_failures_g2[trial, mIndex]     = 0             # Returning NSF means the second constraint was not violated
                seldonian_failures_g3[trial, mIndex]     = 0
                seldonian_fs[trial, mIndex]              = None          # This value should not be used later. We use None and later remove the None values
                print(f"[(worker {worker_id}/{nWorkers}) Seldonian trial {trial+1}/{numTrials}, m {m}] No solution found")

            # Run the Least Squares algorithm
            theta = get_logistic_regression_initial_params(trainX, trainY)                              # Run least squares linear regression
            trueLoss = -fHat(theta, testX, testY)                         # Get the "true" mean squared error using the testData
            LS_failures_g1[trial, mIndex] = 1 if trueLoss > 0.05  else 0   # Check if the first behavioral constraint was violated
            LS_failures_g2[trial, mIndex] = 1 if trueLoss < 1 else 0   # Check if the second behavioral constraint was violated
            LS_failures_g3[trial, mIndex] = 1 if calc_fp_diff(theta, testX, testY) < 0.08 else 0
            
            LS_fs[trial, mIndex] = -trueLoss                              # Store the "true" negative mean-squared error
            print(f"[(worker {worker_id}/{nWorkers}) LeastSq   trial {trial+1}/{numTrials}, m {m}] LS fHat over test data: {trueLoss:.10f}")
        print()

    np.savez(outputFile, 
            ms=ms, 
            seldonian_solutions_found=seldonian_solutions_found,
            seldonian_fs=seldonian_fs, 
            seldonian_failures_g1=seldonian_failures_g1, 
            seldonian_failures_g2=seldonian_failures_g2,
            seldonian_failures_g3=seldonian_failures_g3,
            LS_solutions_found=LS_solutions_found,
            LS_fs=LS_fs,
            LS_failures_g1=LS_failures_g1,
            LS_failures_g2=LS_failures_g2,
            LS_failures_g3=LS_failures_g3)

## Simple comparison with other algoritms

### Simulation

In [ ]:
np.random.seed(1234)  # Create the random number generator to use, with seed zero
numPoints = 5000   # Let's use 5000 points

(X,Y)  = generateData(numPoints)  # Generate the data

train_x, test_x, train_y, test_y = train_test_split(X, Y, random_state=1234)

# Create the behavioral constraints - each is a gHat function and a confidence level delta
gHats  = [g_eq_odds_diff(0.1, batch_size=300)] # The 1st gHat requires MSE < 2.0. The 2nd gHat requires MSE > 1.25
deltas = [0.1]

(result, found) = QSA(train_x, train_y, gHats, deltas) # Run the Quasi-Seldonian algorithm
if found:
  print("Params with constraints", result)
  print("fHat of solution (computed over all data, D):", fHat(result, train_x, train_y))
else:
  print("No solution found")

In [ ]:
def seldonian_predict_factory(theta):
  def _(x):
    return predict(theta, x)
  return _

print(calc_fp_diff(test_x, test_y, predict_prob=seldonian_predict_factory(result)))
print(calc_demographic_parity_diff(test_x, test_y, predict_prob=seldonian_predict_factory(result)))

In [ ]:
from sklearn.metrics import classification_report

y_pred = []
for x in test_x:
  y_pred.append(round(predict(result, [1, *x])))

print(classification_report(test_y, y_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import RandomizedSearchCV

# # Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# # Number of features to consider at every split
# max_features = ['auto', 'sqrt']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# # Method of selecting samples for training each tree
# bootstrap = [True, False]
# # Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}

# rf = RandomForestClassifier()
# clf = RandomizedSearchCV(estimator = rf, 
#                                param_distributions = random_grid, 
#                                n_iter = 100, 
#                                cv = 3, 
#                                verbose=2, 
#                                random_state=42, 
#                                n_jobs = -1)

clf = LogisticRegressionCV(cv=5, random_state=0)

clf.fit(train_x, train_y)

male_indices = test_x[:, 0] == 1
female_indices = test_x[:, 0] == 0

print(
    np.abs(
      calc_fp_sklearn(test_x[male_indices], test_y[male_indices], clf) - 
      calc_fp_sklearn(test_x[female_indices], test_y[female_indices], clf)
    )
)
print(
    np.abs(
      calc_demographic_parity_sklearn(test_x[male_indices], test_y[male_indices], clf) - 
      calc_demographic_parity_sklearn(test_x[female_indices], test_y[female_indices], clf)
    )
)
print(classification_report(test_y, clf.predict(test_x)))

### Real data - Kaggle
https://www.kaggle.com/spscientist/students-performance-in-exams

#### Data processing

In [ ]:
from scipy.stats import zscore

df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Fair AI/german_credit_data.csv')
df['Sex'] = df['Sex'].apply(lambda x: 0 if x == 'female' else 1)

df['Age'] = zscore(df['Age'])
df['Credit amount'] = zscore(df['Credit amount'])
df['Duration'] = zscore(df['Duration'])

df['Job'] = pd.factorize(df['Job'])[0]
df['Housing'] = pd.factorize(df['Housing'])[0]
df['Saving accounts'] = pd.factorize(df['Saving accounts'])[0]
df['Checking account'] = pd.factorize(df['Checking account'])[0]
df['Purpose'] = pd.factorize(df['Purpose'])[0]

# df['Math_PassStatus'] = np.where(df['math score'] < 60, 0, 1)
# df['gender'] = df['gender'].apply(lambda x: 0 if x == 'female' else 1)
# df['race/ethnicity'] = pd.factorize(df['race/ethnicity'])[0]
# df['parental level of education'] = pd.factorize(df['parental level of education'])[0]
# df['lunch'] = pd.factorize(df['lunch'])[0]
# df['test preparation course'] = pd.factorize(df['test preparation course'])[0]
# df['reading score'] = zscore(df['reading score'])
# df['writing score'] = zscore(df['writing score'])
df = df.dropna()
df.head()

#### Modeling

##### Seldonian

In [ ]:
# Create the behavioral constraints - each is a gHat function and a confidence level delta
gHats  = [g_fp_diff(0.025, batch_size=100)]
deltas = [0.5]

# X = df[['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course', 'reading score', 'writing score']].values
X = df[['Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account', 'Age', 'Credit amount', 'Duration', 'Purpose']].values
Y = df['approval'].values

train_x, test_x, train_y, test_y = train_test_split(X, Y, random_state=1234)

(result, found) = QSA(
    train_x, 
    train_y, 
    gHats, 
    deltas
) # Run the Quasi-Seldonian algorithm
if found:
  print("Params with constraints", result)
  print("fHat of solution (computed over all data, D):", fHat(result, X, Y))
else:
  print("No solution found")

In [ ]:
def seldonian_predict_factory(theta):
  def _(x):
    return predict(x, theta)
  return _

print(calc_fp_diff(test_x, test_y, predict_prob=seldonian_predict_factory(result)))
print(calc_demographic_parity_diff(test_x, test_y, predict_prob=seldonian_predict_factory(result)))

In [ ]:
from sklearn.metrics import classification_report

y_pred = []
for x in test_x:
  y_pred.append(round(predict(result, [1, *x])))

print(classification_report(test_y, y_pred))

##### FairLearn

##### Raw model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import classification_report

clf = LogisticRegressionCV(cv=5, random_state=0)
clf.fit(train_x, train_y)

# def rand_forest_predict_prob(x):
#   probs = clf.predict_proba(np.array(x)[1:].reshape((1, -1)))
#   return np.argmax(x)

# calc_fp_diff(X, Y, predict_prob=rand_forest_predict_prob)
male_indices = test_x[:, 0] == 1
female_indices = test_x[:, 0] == 0
print(
    np.abs(
      calc_fp_sklearn(test_x[male_indices], test_y[male_indices], clf) - 
      calc_fp_sklearn(test_x[female_indices], test_y[female_indices], clf)
    )
)
print(
    np.abs(
      calc_demographic_parity_sklearn(test_x[male_indices], test_y[male_indices], clf) - 
      calc_demographic_parity_sklearn(test_x[female_indices], test_y[female_indices], clf)
    )
)
print(classification_report(test_y, clf.predict(test_x)))

### Real data - Algebra Nation

#### Data processing

In [ ]:
import pandas as pd

df_students = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Fair AI/algebra_nation/students.csv')
df_logs = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Fair AI/algebra_nation/logs.csv')
df_sessions = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Fair AI/algebra_nation/sessions.csv')
df_assessments = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Fair AI/algebra_nation/assessments.csv')

In [ ]:
df_students[df_students['grade'] == 12].shape

In [ ]:
df_students_filtered = df_students[~pd.isna(df_students['gender'])]
df_students_filtered.shape

##### Map video id to logs

In [ ]:
log_video_id_map = dict()

for idx, row in df_logs[['id', 'field_name', 'field_val']].iterrows():
  field_name = row.get('field_name')
  log_id = row.get('id')
  if field_name == 'video_id':
    log_video_id_map[log_id] = row.get('field_val')

In [ ]:
df_logs['video_id'] = df_logs['id'].apply(lambda x: log_video_id_map.get(x, None))
df_logs.head()

##### Map user id to logs

In [ ]:
df_logs_merged = df_logs.set_index('session_log_id').join(df_sessions[['id', 'useraccount_id', 'school_id', 'subject_id', 'sub_subject_id', 'os']].set_index('id'))

In [ ]:
df_logs_merged[df_logs_merged.useraccount_id.isin(df_students[df_students['grade'] == 12]['useraccount_id'].unique())].shape

##### Aggregate assessments

In [ ]:
df_assessments.head(1)

In [ ]:
df_assessments_agg = df_assessments.groupby('id')['user_gave_correct_answer'].sum().reset_index().rename(columns={'user_gave_correct_answer': 'num_correct_answer'})
df_assessments_agg = df_assessments_agg.set_index('id')\
  .join(df_assessments[['id', 'useraccount_id', 'session_log_id', 'subject_id', 'section_folder_id', 'number_of_questions', 'is_finished', 'is_adaptive', 'test_type', 'ts_created']]\
  .set_index('id'))\
  .reset_index()\
  .drop_duplicates(['id'])

df_assessments_agg = df_assessments_agg.set_index('id').join(df_assessments.groupby('id')['time_taken'].sum().reset_index().set_index('id')).reset_index()

# merge with sessions
df_sessions_tmp = df_sessions[['id', 'school_id', 'browser', 'os']].set_index('id')
df_assessments_agg = df_assessments_agg.set_index('session_log_id').join(df_sessions_tmp).reset_index().rename(columns={'index': 'session_log_id'})

# merge with students
df_students_tmp = df_students_filtered[['useraccount_id', 'grade', 'hispanic_ethnicity', 'race', 'gender']].set_index('useraccount_id')
df_assessments_agg = df_assessments_agg.set_index('useraccount_id').join(df_students_tmp).reset_index()

df_assessments_agg.head()

##### merge logs to assessments

In [ ]:
df_assessments_agg_filtered = df_assessments_agg.dropna(subset=['subject_id', 'section_folder_id', 'time_taken', 'school_id', 'grade', 'browser', 'os', 'race'])
df_assessments_agg_filtered.shape

In [ ]:
df_logs_merged['ts_created_dt'] = pd.to_datetime(df_logs_merged['ts_created'])

In [ ]:
import numpy as np
from tqdm.auto import tqdm

res = []
memo = {}

actions = df_logs_merged['action_name'].dropna().unique()

for user, createdAt, assess_id in zip(tqdm(df_assessments_agg_filtered['useraccount_id']), df_assessments_agg_filtered['ts_created'], df_assessments_agg_filtered['id']):
  data = df_logs_merged[df_logs_merged['useraccount_id'].isin([user])]
  data = data[(data['ts_created_dt'] <= pd.to_datetime(createdAt))].sort_values('ts_created_dt')
  
  data['duration'] = [*pd.Series(np.diff(data['ts_created_dt'])).apply(lambda x: x.seconds).values, 0]
  data['duration'] = data['duration'].apply(lambda x: x if x <= 60 * 100 else 0)
  frequency = data.groupby(['useraccount_id', 'action_name'])['id'].count().reset_index().rename(columns={'id': 'count'}).set_index('action_name')
  duration = data.groupby(['useraccount_id', 'action_name'])['duration'].sum().reset_index().set_index('action_name')

  assessments = df_assessments_agg_filtered[
    (df_assessments_agg_filtered['useraccount_id'] == user) &
    (df_assessments_agg_filtered['ts_created'] < createdAt)
  ]

  user_res = dict(assess_id=assess_id)
  for action in actions:
    try:
      user_res[f'{action} frequency'] = frequency.loc[action].get('count', 0)
    except:
      user_res[f'{action} frequency'] = 0

    try:
      user_res[f'{action} duration'] = duration.loc[action].get('duration', 0)
    except:
      user_res[f'{action} duration'] = 0

  if assessments.shape[0] > 0:
    user_res['history_avg_num_correct_answer'] = assessments['num_correct_answer'].mean()
    user_res['history_total_num_correct_answer'] = assessments['num_correct_answer'].sum()
    user_res['assessment_order'] = assessments.shape[0] + 1
    user_res['prev_num_correct_answer'] = assessments.tail(1)['num_correct_answer'].values[0]
  else:
    user_res['history_total_num_correct_answer'] = 0
    user_res['history_avg_num_correct_answer'] = 0
    user_res['assessment_order'] = 1
    user_res['prev_num_correct_answer'] = -1

  res.append(user_res)

In [ ]:
df_assessments_agg_filtered = df_assessments_agg_filtered.set_index('id').join(pd.DataFrame(res).set_index('assess_id')).reset_index()
df_assessments_agg_filtered.head(2)

In [ ]:
df_assessments_agg_filtered.to_csv('/content/drive/My Drive/Colab Notebooks/Fair AI/algebra_nation/aggregated_assessments_v5.csv', index=False)

#### Modeling

In [ ]:
df_assessments_agg_filtered = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Fair AI/algebra_nation/aggregated_assessments_v5.csv')

In [ ]:
sample = df_assessments_agg_filtered[df_assessments_agg_filtered['useraccount_id'].isin(pd.Series(df_assessments_agg_filtered['useraccount_id'].unique()).sample(30))]
sample['class_id'] = sample['school_id'].apply(lambda x: 1 if x == 21407.0 else 0)

In [ ]:
sample.to_json('./assessments.json', orient='records')

In [ ]:
!pip install -U plotly kaleido

In [ ]:
subset = df_assessments_agg_filtered[df_assessments_agg_filtered['grade'].isin([12.0])]
subset['race'] = subset['race'].apply(lambda x: 'White' if x == 'Caucasian' else 'Non-white')
subset['gender'] = subset['gender'].apply(lambda x: 'Female' if x == 'f' else 'Male')
subset['pass_status'] = (subset['num_correct_answer'] / subset['number_of_questions']) >= 0.6
subset['pass_status'] = subset['pass_status'].apply(lambda x: 'Passed' if x else 'Failed')

In [ ]:
subset['race'].value_counts()

In [ ]:
subset['gender'].value_counts().reset_index()

In [ ]:
import plotly.express as px

In [ ]:
import plotly.express as px
fig = px.bar(
    subset['race'].replace('Black or African American', 'Blk/African American').replace('Two or More Races', 'Multi-races')\
    .value_counts().reset_index().rename(columns={"index":"race", 'race': "number of assessments"}), 
    x='race', 
    y='number of assessments',
    color='race',
    color_discrete_sequence=['#577590', '#f9c74f', '#f9c74f', '#f9c74f', '#f9c74f'],
)

fig.update_layout(
    showlegend=False,
    font_size=22
)

fig.show()

In [ ]:
fig.write_image("race.png", width=1024, height=768, scale=5)

In [ ]:
import plotly.express as px
fig = px.bar(
    subset['gender'].value_counts().reset_index().replace('f', 'Female').replace('m', 'Male').rename(columns={"index":"gender", 'gender': "number of assessments"}), 
    x='gender', 
    y='number of assessments',
    color='gender',
    color_discrete_sequence=['#ef476f', '#118ab2']
)

fig.update_layout(
    showlegend=False,
    font_size=22
)

fig.show()

In [ ]:
fig.write_image("gender.png", width=1024, height=768, scale=5)

In [ ]:
df_gender_pass = subset.groupby(['gender', 'pass_status'])['id'].count().reset_index().rename(columns={'id': 'count'})
fig = px.bar(
    df_gender_pass, 
    x="gender", 
    y="count", 
    color="pass_status", 
    barmode="group", 
    color_discrete_sequence=['#ef476f', '#118ab2']
)

fig.update_layout(
    font_size=22
)

fig.show()

In [ ]:
fig.write_image("gender_pass_status.png", width=1024, height=768, scale=4)

In [ ]:
df_gender_pass = subset.groupby(['race', 'pass_status'])['id'].count().reset_index().rename(columns={'id': 'count'})
fig = px.bar(
    df_gender_pass, 
    x="race",
    y="count", 
    color="pass_status", 
    barmode="group", 
    color_discrete_sequence=['#ef476f', '#118ab2']
  )
fig.update_layout(
    font_size=22
)
fig.show()

In [ ]:
fig.write_image("race_pass_status.png", width=1024, height=768, scale=4)

In [ ]:
from sklearn import preprocessing

df_assessments_agg_filtered = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Fair AI/algebra_nation/aggregated_assessments_v5.csv')
df_logs = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Fair AI/algebra_nation/logs.csv')

# filtering
df_assessments_agg_filtered = df_assessments_agg_filtered[df_assessments_agg_filtered['grade'].isin([12.0])]
df_assessments_agg_filtered = df_assessments_agg_filtered[df_assessments_agg_filtered['is_finished'] == 1]

# available_actions = df_logs['action_name'].dropna().unique()
available_actions = [
  'video_watch', 'video_pause', 'video_play', 
  'video_seek', 'video_completed', 
  'tys_review_correct_question', 'tys_review_solution_video', 'tys_review_topic_video',
  # 'wall_make_post', 'wall_search'
]

df_assessments_agg_filtered['pass_status'] = \
  (df_assessments_agg_filtered['num_correct_answer'] / df_assessments_agg_filtered['number_of_questions']) >= 0.6

numeric_cols = []
numeric_cols = [f'{action} frequency' for action in available_actions]
# numeric_cols = [*numeric_cols, *[f'{action} duration' for action in available_actions]]
numeric_cols = ['history_avg_num_correct_answer', 'assessment_order', *numeric_cols]

factor_cols = [
  'race',
  'gender',
  # 'grade',
  'subject_id', 
  'school_id', 
  # 'browser', 'os',
]
wanted_cols = [
  *factor_cols,
  *numeric_cols
]

for col in factor_cols:
  if col == 'gender':
    df_assessments_agg_filtered[col] = df_assessments_agg_filtered[col].apply(lambda x: 1 if x == 'm' else 0)
  if col == 'race':
    df_assessments_agg_filtered[col] = df_assessments_agg_filtered[col].apply(lambda x: 1 if x == 'Black or African American' else 0)
  else:
    df_assessments_agg_filtered[col] = pd.factorize(df_assessments_agg_filtered[col])[0]

df_assessments_agg_filtered[numeric_cols] = preprocessing.scale(df_assessments_agg_filtered[numeric_cols])

df_assessments_agg_filtered['pass_status_factor'] = df_assessments_agg_filtered['pass_status'].apply(lambda x: 1 if x else 0)

df_assessments_agg_filtered[[*wanted_cols, 'pass_status_factor']].head()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split

grade_predictor = SVC(gamma='auto')

X = df_assessments_agg_filtered[wanted_cols].values
Y = df_assessments_agg_filtered['num_correct_answer'].values

train_x, test_x, train_y, test_y = train_test_split(X, Y, random_state=1234)

grade_predictor.fit(train_x, train_y)
print(grade_predictor.score(train_x, train_y))

df_assessments_agg_filtered['predicted_num'] = grade_predictor.predict(X)
df_assessments_agg_filtered['predicted_num'] = preprocessing.scale(df_assessments_agg_filtered['predicted_num'])

##### QSA

In [ ]:
X = df_assessments_agg_filtered[[*wanted_cols, 'predicted_num']].values
Y = df_assessments_agg_filtered['pass_status_factor'].values

train_x, test_x, train_y, test_y = train_test_split(X, Y, random_state=1234)

In [ ]:
# gHats  = [g_eq_odds_diff(0.03, batch_size=150, bootstrap_num=300)]
gHats  = [g_max_loss(0.15), g_eq_odds_diff(0.001, batch_size=200, bootstrap_num=64)]
deltas = [0.1, 0.1]

(result, found) = QSA(
    train_x,
    train_y,
    gHats,
    deltas
) # Run the Quasi-Seldonian algorithm
if found:
  print("Params with constraints", result)
  print("fHat of solution (computed over all data, D):", fHat(result, X, Y))
else:
  print("No solution found")

```python
# 12th grade

####################START OF GENDER#####################

# (1)
###############
gHats  = [g_max_loss(0.15), g_eq_odds_diff(0.01, batch_size=200, bootstrap_num=64)]
deltas = [0.1, 0.1]
results = [-0.49422,  0.57848, -1.47481, -0.19909, -0.00524,  0.62251,
        0.1279 ,  0.29416,  0.25328, -0.32137,  0.20153, -0.24534,
       -0.08981, -0.15302,  0.02212,  0.63059]
eq_odds = 0.0726
###############

# (2)
###############
gHats  = [g_max_loss(0.15), g_eq_odds_diff(0.05, batch_size=200, bootstrap_num=64)]
deltas = [0.1, 0.1]

results = [
  -0.49422,  0.57848, -1.47481, -0.19916, -0.00747,  0.61705,
  -0.10081,  0.35104,  0.25394, -0.32137,  0.20153, -0.24534,
  -0.17246, -0.03464,  0.02665,  0.61185
]

# gender
eq_odds = 0.08564
###############

# (3)
###############
###############

####################END OF GENDER#####################


####################START OF RACE#####################

###############
g_eq_odds_diff(0.25)
results = [
  -1.4812 , -0.07405,  0.16733, -0.01593, -0.00515,  0.6227 ,
  -0.024  ,  0.29256,  0.2464 , -0.3248 ,  0.22267, -0.25594,
  -0.08992,  0.04587,  0.02941,  0.63061
]

# race
eq_odds = 0.2082
###############

###############
g_eq_odds_diff(0.2)
results = [
  -1.62982,  0.16682,  0.10559,  0.11313, -0.00512,  0.32857,
  -0.06024,  0.28862,  0.2536 , -0.32152,  0.21349, -0.24508,
  -0.09937,  0.09117,  0.02944,  0.63061
]

# race
eq_odds = 0.17937
###############

###############
g_eq_odds_diff(0.125)
results = [
  -1.4812 , -0.08077,  0.10554, -0.00644, -0.00515,  0.61669,
  -0.02142,  0.29283,  0.25304, -0.32152,  0.20212, -0.24574,
  -0.08625,  0.04564,  0.02938,  0.08865
]

# race
eq_odds = 0.11936
###############

###############
g_eq_odds_diff(0.05)
results = [
  -2.45165, -0.37855,  1.06118,  0.02282, -0.00513,  0.25044,
  0.13393,  0.18211,  0.25874, -0.32149,  0.20146, -0.24537,
  -0.08767,  0.06238,  0.02938,  0.63061
]

# race
eq_odds = 0.0673
###############
```



In [ ]:
def seldonian_predict_factory(theta):
  def _(x):
    return predict(x, theta)
  return _

print(calc_eq_odds_diff(test_x, test_y, predict_prob=seldonian_predict_factory(result)))
print(calc_eq_op_diff(test_x, test_y, predict_prob=seldonian_predict_factory(result)))

In [ ]:
race_results = [
  [
    -1.4812 , -0.07405,  0.16733, -0.01593, -0.00515,  0.6227 ,
    -0.024  ,  0.29256,  0.2464 , -0.3248 ,  0.22267, -0.25594,
    -0.08992,  0.04587,  0.02941,  0.63061
  ],
  [
    -1.62982,  0.16682,  0.10559,  0.11313, -0.00512,  0.32857,
    -0.06024,  0.28862,  0.2536 , -0.32152,  0.21349, -0.24508,
    -0.09937,  0.09117,  0.02944,  0.63061
  ],
  [
    -1.4812 , -0.08077,  0.10554, -0.00644, -0.00515,  0.61669,
    -0.02142,  0.29283,  0.25304, -0.32152,  0.20212, -0.24574,
    -0.08625,  0.04564,  0.02938,  0.08865
  ],
  [
    -2.45165, -0.37855,  1.06118,  0.02282, -0.00513,  0.25044,
    0.13393,  0.18211,  0.25874, -0.32149,  0.20146, -0.24537,
    -0.08767,  0.06238,  0.02938,  0.63061
  ]
]

gender_results = [
  [
    -0.49422,  0.57848, -1.47481, -0.19909, -0.00524,  0.62251,
    0.1279 ,  0.29416,  0.25328, -0.32137,  0.20153, -0.24534,
    -0.08981, -0.15302,  0.02212,  0.63059
  ],
  [
    -0.49422,  0.57848, -1.47481, -0.19916, -0.00747,  0.61705,
    -0.10081,  0.35104,  0.25394, -0.32137,  0.20153, -0.24534,
    -0.17246, -0.03464,  0.02665,  0.61185
  ]
]

In [ ]:
from sklearn.metrics import classification_report

def seldonian_predict_factory(theta):
  def _(x):
    return predict(x, theta)
  return _

for result in race_results:
  y_pred = []
  y_pred_prob = []
  for x in test_x:
    y_pred.append(round(predict(result, [1, *x])))
    y_pred_prob.append(predict(result, [1, *x]))

  print(calc_eq_odds_diff(test_x, test_y, predict_prob=seldonian_predict_factory(result)))
  print(classification_report(test_y, y_pred))
  fpr, tpr, thresholds = roc_curve(test_y, y_pred_prob)
  print(auc(fpr, tpr))
  

###### Explanation

In [ ]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
import numpy as np

# race - 0.25
result = [
  -1.4812 , -0.07405,  0.16733, -0.01593, -0.00515,  0.6227 ,
  -0.024  ,  0.29256,  0.2464 , -0.3248 ,  0.22267, -0.25594,
  -0.08992,  0.04587,  0.02941,  0.63061
]
clf = LogisticRegression(random_state=1234)
clf.fit(train_x, train_y)
clf.coef_ = np.array([result[1:]])
clf.intercept_ = np.array([result[0]])

from sklearn.ensemble import RandomForestClassifier

# clf = RandomForestClassifier(random_state=1234)
# clf = LogisticRegressionCV(cv=5, random_state=0, max_iter=10000)
# clf.fit(train_x, train_y)

In [ ]:
import dalex as dx

data = pd.DataFrame(train_x, columns=[*wanted_cols, 'predicted_num'])

exp = dx.Explainer(clf, data, train_y)

In [ ]:
stu = data.loc[0,:]

bd_stu = exp.predict_parts(stu, type='break_down')
bd_interactions_stu = exp.predict_parts(stu, type='break_down_interactions')
sh_stu = exp.predict_parts(stu, type='shap', B=10)

In [ ]:
bd_stu.result.label = "Student 11"
bd_interactions_stu.result.label = "Student 11+"

bd_stu.plot(bd_interactions_stu, max_vars=30)

In [ ]:
sh_stu.result.label = "Student 11"

sh_stu.plot(bar_width = 16, max_vars=30)

In [ ]:
vi = exp.model_parts()

vi.result.label = "Model"

vi.plot(max_vars=30)

In [ ]:
pdp_num = exp.model_profile(type = 'partial')
pdp_num.result["_label_"] = 'Model'

pdp_num.plot()

###### Eq Odds trend

In [ ]:
race_performance = pd.DataFrame([
  {'Required Max Equalized odds': 0.25, 'F1': 0.78},
  {'Required Max Equalized odds': 0.20, 'F1': 0.77},
  {'Required Max Equalized odds': 0.15, 'F1': 0.72},
  {'Required Max Equalized odds': 0.10, 'F1': 0.69},
])

import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=race_performance['Required Max Equalized odds'], y=race_performance['F1'],
                    mode='lines+markers',
                    name='lines+markers')
)
# fig = px.scatter(race_performance, x="Equalized odds", y="F1", title='Fairness & Performance Trade-off (Race)')
fig.update_layout(
    xaxis_title="Required Max Equalized Odds",
    yaxis_title="F-measure",
    font=dict(
        size=16,
        color="#7f7f7f"
    )
)
fig.show()

In [ ]:
fig.write_image("perf_fair_tradeoff.png", scale=4)

###### Model fairness comparison

In [ ]:
import pandas as pd

df_performance = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Fair AI/model_performance.csv')
df_performance['Model'] = df_performance['Model'].str.replace('SA_odds_', 'Fair-LR@')

# adjust the order of eo@0.01 and eo@0.05
temp = df_performance.iloc[11].copy()
df_performance.iloc[11] = df_performance.iloc[10]
df_performance.iloc[10] = temp

df_performance

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

colors = {
  "race": ['#7CAFC4', '#7CAFC4', '#7CAFC4', '#5995ED', '#5995ED', '#5995ED', '#5995ED'],
  "gender": ['#7CAFC4', '#7CAFC4', '#7CAFC4', '#5995ED', '#5995ED']
}
fig = make_subplots(rows=1, cols=2, subplot_titles=("Model Fairness - Race", "Model Fairness - Gender"))

for idx, group in enumerate(df_performance['Group'].unique()):
  data = df_performance[df_performance['Group'] == group]
  fig.add_trace(
    go.Bar(
    x=data['Model'],
    y=data['Equalized Odds'],
    marker_color=colors[group] # marker color can be a single color value or an iterable
  ), row=1, col=idx+1)

fig.update_layout(
    showlegend=False,
    font_size=16
)
fig.show()

In [ ]:
fig.write_image("model_comparison.png", width=1400, height=768, scale=3)

###### Decision boundary

https://mc.ai/decision-boundary-plotting-for-high-dimensional-data/

In [ ]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

# race
# result = [
#   -1.4812 , -0.07405,  0.16733, -0.01593, -0.00515,  0.6227 ,
#   -0.024  ,  0.29256,  0.2464 , -0.3248 ,  0.22267, -0.25594,
#   -0.08992,  0.04587,  0.02941,  0.63061
# ]

# gender
# result = [
#   -0.49422,  0.57848, -1.47481, -0.19909, -0.00524,  0.62251,
#   0.1279 ,  0.29416,  0.25328, -0.32137,  0.20153, -0.24534,
#   -0.08981, -0.15302,  0.02212,  0.63059
# ]
# clf = LogisticRegression(random_state=1234)
# clf.fit(train_x, train_y)
# clf.coef_ = np.array([result[1:]])
# clf.intercept_ = np.array([result[0]])

clf = LogisticRegressionCV(cv=5, random_state=0, max_iter=10000)
clf.fit(train_x, train_y)

In [ ]:
from sklearn.manifold import TSNE# set random seed
seed = 1
np.random.seed(seed) # transform original data into 2D

rand_indice = np.random.randint(train_x.shape[0], size=500)
# plot_x = pd.DataFrame(train_x[rand_indice, :]).append(pd.DataFrame(test_x)).values
# plot_y = pd.DataFrame(train_y[rand_indice]).append(pd.DataFrame(test_y)).values

plot_x = test_x.copy()
plot_y = test_y.copy()

race = plot_x[:, 0]
tsne = TSNE(n_components=2)
X_2d = tsne.fit_transform(plot_x) # shape: (num_samples, 2)# split original and transformed data using same random seed

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier# build and train original model

X_train, X_test, y_train, y_test = train_test_split(plot_x, plot_y, train_size=0.99, random_state=seed)
X_train_2d, X_test_2d, y_train, y_test = train_test_split(X_2d, plot_y, train_size=0.99, random_state=seed)

y_pred = clf.predict(X_train)

voronoi = KNeighborsClassifier(n_neighbors=1)
voronoi.fit(X_train_2d, y_pred)

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.ticker import NullFormatter

# 1. generate 2D background grid
pad = 0.5
min_x1, max_x1 = np.min(X_train_2d[:, 0]) - pad, np.max(X_train_2d[:, 0]) + pad
min_x2, max_x2 = np.min(X_train_2d[:, 1]) - pad, np.max(X_train_2d[:, 1]) + pad

def generate_grid_points(min_x, max_x, min_y, max_y, resolution=0.1):
    """Generate resolution * resolution points within a given range."""
    return np.meshgrid(
        np.arange(min_x, max_x, resolution),
        np.arange(min_y, max_y, resolution)
    )
    
xx, yy = generate_grid_points(min_x1, max_x1, min_x2, max_x2)

# 2. get model's predictions for grid
background = voronoi.predict(np.c_[xx.ravel(), yy.ravel()])
background = background.reshape(xx.shape)

# plot data along with decision boundary
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
colors = ['#FF0064', '#0839FF']
cmap = ListedColormap(['#FF0064', '#0839FF'])

# 3. plot grid with predictions (this forms the decision boundary)
ax.contourf(xx, yy, background, alpha=0.4, cmap=cmap)

protected_attribute = 'gender'
col_idx = 0 if protected_attribute == 'gender' else 1
# 4. plot training and test data
for idx, sensitive in enumerate(X_train[:, col_idx]):
  ax.scatter(
      X_train_2d[idx, 0], 
      X_train_2d[idx, 1], 
      c=colors[y_train[idx]],
      marker="P" if sensitive else '.', 
      s=200,
      zorder=10,
      edgecolor='#f8f8f8', 
      linewidth=0.75, 
  )
# ax.scatter(X_test_2d[:, 0], X_test_2d[:, 1], c=y_test,
#            cmap=cmap, marker="x", label="Test")

# other specifications
# ax.set_title("Logistic Regression")
ax.set_xlim([min_x1, max_x1])
ax.set_ylim([min_x2, max_x2])

# https://stackoverflow.com/a/18861258
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
ax.xaxis.set_ticks_position('none')
ax.yaxis.set_ticks_position('none')

# Don't allow the axis to be on top of your data
# ax.set_axisbelow(True)
ax.minorticks_on()
# Customize the major grid
ax.grid(which='major', linestyle='-', linewidth='1', color='#f8f8f8')
# Customize the minor grid
ax.grid(which='minor', linestyle='-', linewidth='1', color='#f8f8f8')

# ax.legend(loc="best", title="Data")

%config InlineBackend.figure_format = 'retina'

plt.show()

In [ ]:
!pip install scikit-image

In [ ]:
from skimage.measure import compare_ssim
import cv2
import numpy as np

before = cv2.imread('./nonfair-gender.png')
after = cv2.imread('./fair-gender.png')

# Convert images to grayscale
before_gray = cv2.cvtColor(before, cv2.COLOR_BGR2GRAY)
after_gray = cv2.cvtColor(after, cv2.COLOR_BGR2GRAY)

# Compute SSIM between two images
(score, diff) = compare_ssim(before_gray, after_gray, full=True)
print("Image similarity", score)

# The diff image contains the actual image differences between the two images
# and is represented as a floating point data type in the range [0,1] 
# so we must convert the array to 8-bit unsigned integers in the range
# [0,255] before we can use it with OpenCV
diff = (diff * 255).astype("uint8")

# Threshold the difference image, followed by finding contours to
# obtain the regions of the two input images that differ
thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

mask = np.zeros(before.shape, dtype='uint8')
filled_after = after.copy()

for c in contours:
    area = cv2.contourArea(c)
    if area > 40:
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(before, (x, y), (x + w, y + h), (36,255,12), 2)
        cv2.rectangle(after, (x, y), (x + w, y + h), (36,255,12), 2)
        cv2.drawContours(mask, [c], 0, (0,255,0), -1)
        cv2.drawContours(filled_after, [c], 0, (0,255,0), -1)

# cv2.imshow('before', before)
# cv2.imshow('after', after)
# cv2.imshow('diff',diff)
# cv2.imshow('mask',mask)
# cv2.imshow('filled after',filled_after)
cv2.waitKey(0)

In [ ]:
cv2.imwrite('./gender_compared.png', filled_after)

##### ML

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc
from sklearn.svm import SVC

# clf = LogisticRegressionCV(cv=5, random_state=0, max_iter=10000)
# clf = RandomForestClassifier(random_state=1234)
clf = SVC(gamma='auto', probability=True, random_state=1234)
clf.fit(train_x, train_y)

male_indices = test_x[:, 0] == 1
female_indices = test_x[:, 0] == 0

y_pred = clf.predict(test_x)
print("Equalized odds:", calc_odds_sklearn(test_x, test_y, clf))
print("Equal Opportunity:", calc_eq_op_sklearn(test_x, test_y, clf))
print(classification_report(test_y, clf.predict(test_x)))

fpr, tpr, thresholds = roc_curve(test_y, clf.predict_proba(test_x)[:, 1])
print(auc(fpr, tpr))

In [ ]:
2*((0.77*0.71)/(0.77+0.71))

##### Fair-learn

In [ ]:
from fairlearn.widget import FairlearnDashboard
from sklearn.model_selection import train_test_split
from fairlearn.reductions import GridSearch
from fairlearn.reductions import DemographicParity, ErrorRate, EqualizedOdds

sweep = GridSearch(
    LogisticRegressionCV(cv=5, random_state=0, max_iter=10000),
    constraints=EqualizedOdds(),
    grid_size=71
)

In [ ]:
non_sensitive_cols = [col for col in wanted_cols if col != 'race']
X = df_assessments_agg_filtered[[*non_sensitive_cols, 'predicted_num']].values
Y = df_assessments_agg_filtered['pass_status_factor'].values
A = df_assessments_agg_filtered.race.values

X_train, X_test, Y_train, Y_test, A_train, A_test = train_test_split(
    X,
    Y,
    A,
    random_state=1234,
    stratify=Y
)

In [ ]:
sweep.fit(X_train, Y_train, sensitive_features = A_train)

In [ ]:
y_pred = sweep.predict(X_test)

print(classification_report(Y_test, y_pred))

In [ ]:
from fairlearn.metrics import equalized_odds_difference

equalized_odds_difference(Y_test, y_pred, sensitive_features=A_test)

In [ ]:
errors, eq_odds = [], []
for m in sweep._predictors:
    def classifier(X): return m.predict(X)

    error = ErrorRate()
    error.load_data(X_train, pd.Series(Y_train), sensitive_features=A_train)

    odds = EqualizedOdds()
    odds.load_data(X_train, pd.Series(Y_train), sensitive_features=A_train)
    
    errors.append(error.gamma(classifier)[0])
    eq_odds.append(odds.gamma(classifier).max())

all_results = pd.DataFrame({"predictor": sweep._predictors, "equalized_odds": eq_odds, 'error': errors})

non_dominated = []
for row in all_results.itertuples():
    errors_for_lower_or_eq_disparity = all_results["error"][all_results["equalized_odds"] <= row.equalized_odds]
    if row.error <= errors_for_lower_or_eq_disparity.min():
        non_dominated.append(row.predictor)

In [ ]:
dashboard_predicted = {}

for i in range(len(non_dominated)):
    key = "dominant_model_{0}".format(i)
    value = non_dominated[i].predict(X_test)
    dashboard_predicted[key] = value


FairlearnDashboard(sensitive_features=A_test, sensitive_feature_names=['race'],
                   y_true=Y_test,
                   y_pred=dashboard_predicted)

## Run experiment

In [ ]:
ray.shutdown()

In [ ]:
ray.shutdown()
ray.init()

# Create the behavioral constraints - each is a gHat function and a confidence level delta
gHats  = [gHat1, gHat2, gHat3] # The 1st gHat requires MSE < 2.0. The 2nd gHat requires MSE > 1.25
deltas = [0.1, 0.1, 0.1]
nWorkers = 8

# We will use different amounts of data, m. The different values of m will be stored in ms.
# These values correspond to the horizontal axis locations in all three plots we will make.
# We will use a logarithmic horizontal axis, so the amounts of data we use shouldn't be evenly spaced.
ms   = [2**i for i in range(5, 17)]  # ms = [32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384, 32768, 65536]
numM = len(ms)

# How many trials should we average over?
numTrials = 70 # We pick 70 because with 70 trials per worker, and 16 workers, we get >1000 trials for each value of m

# How much data should we generate to compute the estimates of the primary objective and behavioral constraint function values 
# that we call "ground truth"? Each candidate solution deemed safe, and identified using limited training data, will be evaluated 
# over this large number of points to check whether it is really safe, and to compute its "true" mean squared error.
mTest = ms[-1] * 100 # about 5,000,000 test samples

# Start 'nWorkers' threads in parallel, each one running 'numTrials' trials. Each thread saves its results to a file
tic = timeit.default_timer()
_ = ray.get([run_experiments.remote(worker_id, nWorkers, ms, numM, numTrials, mTest) for worker_id in range(1,nWorkers+1)])
toc = timeit.default_timer()
time_parallel = toc - tic # Elapsed time in seconds
print(f"Time ellapsed: {time_parallel}")

In [ ]:
(X[:, 1] == 0).sum()